In [1]:
import pandas as pd
import os
#SQL Alchemy
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import func
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [2]:
path_measurements = os.path.join('Resources', 'clean_hawaii_measurements.csv')
path_stations= os.path.join('Resources', 'clean_hawaii_stations.csv')

In [3]:
df_measure = pd.read_csv(path_measurements, dtype=object)
df_measure.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0,63
2,USC00519397,2010-01-03,0,74
3,USC00519397,2010-01-04,0,76
4,USC00519397,2010-01-07,0.06,70


In [16]:
# Read CSV file into a pandas DataFrame
df_stations = pd.read_csv(path_stations, dtype=object)
df_stations.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [17]:
# Create an engine to a database file called `demographics.sqlite`
### BEGIN SOLUTION
engine = create_engine("sqlite:///hawaii.sqlite")
### END SOLUTION

In [18]:
# Create a connection to the engine called `conn`
### BEGIN SOLUTION
conn = engine.connect()
### END SOLUTION

In [19]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column

Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'Measurement'

    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Integer)

    def __repr__(self):
        return f"id={self.id}, station={self.station}"

In [20]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column

class Station(Base):
    __tablename__ = 'Station'

    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [21]:
# Use `create_all` to create the customers table in the database
Base.metadata.create_all(engine)

In [22]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
measurement_data = df_measure.to_dict(orient='records')
measurement_data[0]

{'date': '2010-01-01', 'prcp': '0.08', 'station': 'USC00519397', 'tobs': '65'}

In [23]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
station_data = df_stations .to_dict(orient='records')
station_data[0]

{'elevation': '3',
 'latitude': '21.2716',
 'longitude': '-157.8168',
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [24]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [25]:
# Save the reference to the `Measurements` table as a variable called `measure_tbl`
measure_tbl = sqlalchemy.Table('Measurement', metadata, autoload=True)

In [26]:
# Save the reference to the `Station` table as a variable called `station_tbl`
station_tbl = sqlalchemy.Table('Station', metadata, autoload=True)

In [27]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(measure_tbl.delete())
conn.execute(station_tbl.delete())

In [28]:
# Use `measure_tbl.insert()` and `station_tbl.insert()` to insert the data into the tables

conn.execute(measure_tbl.insert(), measurement_data)
conn.execute(station_tbl.insert(), station_data)

In [29]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from Measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]

In [30]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from Station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]